In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install -U transformers
!pip install -U datasets
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U bitsandbytes
!pip install -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.3 MB/s eta 0:00:00:00:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 14.5 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 3.0.1
    Uninstalling datasets-3.0.1:
      Successfully uninstalled datasets-3.0.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 10.6 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 8.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 8.5 MB/s eta 0

In [ ]:
!pip install torch torchvision torchaudio

In [ ]:
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from peft import(
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)

import os,torch,wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

In [10]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
import re

In [ ]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token= user_secrets.get_secret("ResumeParser")
login(token=hf_token)

wb_token = user_secrets.get_secret("wandb")
wandb.login(key=wb_token)

run=wandb.init(
    project='Fine tuning llama 3 8B on Resume dataset',
    job_type='training',
    anonymous="allow"
)

In [ ]:
base_model='/kaggle/input/llama-3/transformers/8b-chat-hf/1'
dataset_name='/kaggle/input/resume-dataset/processedresume.csv'
new_model="llama-3-8b-ResumeParser"

In [1]:
torch_dtype=torch.float16
attn_implementation="eager"

NameError: name 'torch' is not defined

In [ ]:
bnb_config= BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch_dtype,
    nb_4bit_use_double_quant=True,
)

model= AutoModelforCasualLLM.from_pretrained(
    base_model,
    quantization_config= bnb_config,
    num_labels=55,
    device_map='auto',
    attn_implementation=attn_implementation
)

In [ ]:
tokenizer= AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token_id = tokenizer.eos_token_id
model,tokenizer=setup_chat_format(model,tokenizer)

In [ ]:
peft_config=LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.5,
    bias="none",
    task_type='SEQ_CLS',
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)

model = get_peft_model(model, peft_config)

In [5]:
df=pd.read_csv("/kaggle/input/resume-dataset/processedresume.csv")
print(df['Category'].unique())
print(df.shape[0])

['Data Science' 'HR' 'Advocate' 'Arts' 'Web Designing'
 'Mechanical Engineer' 'Sales' 'Health and fitness' 'Civil Engineer'
 'Java Developer' 'Business Analyst' 'SAP Developer' 'Automation Testing'
 'Electrical Engineering' 'Operations Manager' 'Python Developer'
 'DevOps Engineer' 'Network Security Engineer' 'PMO' 'Database' 'Hadoop'
 'ETL Developer' 'DotNet Developer' 'Blockchain' 'Testing'
 ' Process Improvements' 'DESIGNER' ' Word' 'INFORMATION-TECHNOLOGY'
 ' Oracle database' 'TEACHER' 'ADVOCATE' ' Business Development'
 'BUSINESS-DEVELOPMENT'
 ' worked closely with end customer and subcontractors (Solectron'
 'HEALTHCARE' 'FITNESS' 'AGRICULTURE' 'BPO' ' and cobra' 'SALES'
 'CONSULTANT' 'DIGITAL-MEDIA' 'AUTOMOBILE' ' CIM 7.0' 'CHEF' 'FINANCE'
 'APPAREL' 'ENGINEERING' 'ACCOUNTANT' 'CONSTRUCTION' 'PUBLIC-RELATIONS'
 'BANKING' 'ARTS' 'AVIATION']
3427


In [17]:
def cleaning(text):
    text = re.sub(r"(https?://[^\s]+)", "", text)
    text = re.sub(r"(?:RT|cc|#\S+|@\S+)", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(r"[^\x00-\x7F]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df['cleaned'] = df['Resume'].apply(lambda x: cleaning(str(x)))
df['cleaned'] = df['cleaned'].str.lower()

In [18]:
print(df.head())

       Category                                             Resume Unnamed: 2  \
0  Data Science  Skills * Programming Languages: Python (pandas...        NaN   
1  Data Science  Education Details \r\nMay 2013 to May 2017 B.E...        NaN   
2  Data Science  Areas of Interest Deep Learning, Control Syste...        NaN   
3  Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...        NaN   
4  Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...        NaN   

   Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8  \
0         NaN        NaN        NaN        NaN        NaN        NaN   
1         NaN        NaN        NaN        NaN        NaN        NaN   
2         NaN        NaN        NaN        NaN        NaN        NaN   
3         NaN        NaN        NaN        NaN        NaN        NaN   
4         NaN        NaN        NaN        NaN        NaN        NaN   

  Unnamed: 9  ... Unnamed: 155 Unnamed: 156 Unnamed: 157 Unnamed: 158  \
0      

In [22]:
df = df.dropna(axis=1, how='any')

print("\nDataFrame after dropping empty columns:")
print(df)


DataFrame after dropping empty columns:
          Category                                             Resume  \
0     Data Science  Skills * Programming Languages: Python (pandas...   
1     Data Science  Education Details \r\nMay 2013 to May 2017 B.E...   
2     Data Science  Areas of Interest Deep Learning, Control Syste...   
3     Data Science  Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...   
4     Data Science  Education Details \r\n MCA   YMCAUST,  Faridab...   
...            ...                                                ...   
3422      AVIATION           RANK: SGT/E-5 NON- COMMISSIONED OFFIC...   
3423      AVIATION           GOVERNMENT RELATIONS, COMMUNICATIONS ...   
3424      AVIATION           GEEK SQUAD AGENT         Professional...   
3425      AVIATION           PROGRAM DIRECTOR / OFFICE MANAGER    ...   
3426      AVIATION           STOREKEEPER II       Professional Sum...   

                                                cleaned  
0     skills programming

In [19]:
df= df.sample(frac=1,random_state=85).reset_index(drop=True).head(3000)

train_size=0.8
test_size=0.1

train_end = int(train_size * len(df))
test_end = train_end + int(test_size * len(df))

X_train = df[:train_end]
X_test = df[train_end:test_end]
X_eval = df[test_end:]

def generate_prompt(data_point):
    return f"""
            Classify the resume into Data Science, Software Engineering, Project Management, etc.
resume_text: {data_point["cleaned"]}
category: {data_point["Category"]}""".strip()

def generate_test_prompt(data_point):
    return f"""
            Classify the resume into Data Science, Software Engineering, Project Management, etc.
resume_text: {data_point["cleaned"]}
Category: """.strip()

X_train = X_train.copy()
X_eval = X_eval.copy()

# Generate prompts for training and evaluation data
X_train.loc[:, 'text'] = X_train.apply(generate_prompt, axis=1)
X_eval.loc[:, 'text'] = X_eval.apply(generate_prompt, axis=1)


# Generate test prompts and extract true labels
y_true = X_test['Category']
X_test = pd.DataFrame(X_test.apply(generate_test_prompt, axis=1), columns=["text"])


In [38]:
X_train.Category.value_counts()

Category
HR                                                                 94
AVIATION                                                           77
ADVOCATE                                                           75
ENGINEERING                                                        72
ACCOUNTANT                                                         72
HEALTHCARE                                                         72
FITNESS                                                            72
TEACHER                                                            71
INFORMATION-TECHNOLOGY                                             69
CONSULTANT                                                         69
PUBLIC-RELATIONS                                                   68
BANKING                                                            68
SALES                                                              68
CHEF                                                               67
FINANCE    

In [39]:
train_data = Dataset.from_pandas(X_train[["cleaned"]])
eval_data = Dataset.from_pandas(X_eval[["cleaned"]])

NameError: name 'Dataset' is not defined

In [8]:
print(df['Category'].unique().tolist())

['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing', 'Mechanical Engineer', 'Sales', 'Health and fitness', 'Civil Engineer', 'Java Developer', 'Business Analyst', 'SAP Developer', 'Automation Testing', 'Electrical Engineering', 'Operations Manager', 'Python Developer', 'DevOps Engineer', 'Network Security Engineer', 'PMO', 'Database', 'Hadoop', 'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing', ' Process Improvements', 'DESIGNER', ' Word', 'INFORMATION-TECHNOLOGY', ' Oracle database', 'TEACHER', 'ADVOCATE', ' Business Development', 'BUSINESS-DEVELOPMENT', ' worked closely with end customer and subcontractors (Solectron', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE', 'BPO', ' and cobra', 'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE', ' CIM 7.0', 'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT', 'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION']


In [20]:
def predict(test,model,tokenizer):
    y_pred=[]
    Category=['Data Science', 'HR', 'Advocate', 'Arts', 'Web Designing', 'Mechanical Engineer', 'Sales', 'Health and fitness', 'Civil Engineer', 'Java Developer', 'Business Analyst', 'SAP Developer', 'Automation Testing', 'Electrical Engineering', 'Operations Manager', 'Python Developer', 'DevOps Engineer', 'Network Security Engineer', 'PMO', 'Database', 'Hadoop', 'ETL Developer', 'DotNet Developer', 'Blockchain', 'Testing', ' Process Improvements', 'DESIGNER', 'INFORMATION-TECHNOLOGY', ' Oracle database', 'TEACHER', 'ADVOCATE', ' Business Development', 'BUSINESS-DEVELOPMENT', 'HEALTHCARE', 'FITNESS', 'AGRICULTURE', 'BPO',  'SALES', 'CONSULTANT', 'DIGITAL-MEDIA', 'AUTOMOBILE', 'CHEF', 'FINANCE', 'APPAREL', 'ENGINEERING', 'ACCOUNTANT', 'CONSTRUCTION', 'PUBLIC-RELATIONS', 'BANKING', 'ARTS', 'AVIATION']
    for i in tqdm(range(len(test))):
        prompt=test.iloc[i]["cleaned"]
        pipe=pipeline(task="text generation",
                      model=model,
                      tokenizer=tokenizer,
                      max_new_tokens=2,
                      temperature=0.1)
    for i in Category:
         if category.lower() in answer.lower():
                y_pred.append(category)
                break
         else:
            y_pred.append("none")
    
    return y_pred

y_pred = predict(X_test, model, tokenizer)

NameError: name 'model' is not defined